In [27]:
import os
import pandas as pd
import argparse
import requests
import json
from dotenv import load_dotenv
from pathlib import Path
load_dotenv()
PROJECT_ROOT = Path.cwd()
BASE_URL = 'https://datawebws.usitc.gov/dataweb'
requests.packages.urllib3.disable_warnings() 
basicQuery = {
    "savedQueryName":"",
    "savedQueryDesc":"",
    "isOwner":True,
    "runMonthly":False,
    "reportOptions":{
        "tradeType":"Import",
        "classificationSystem":"HTS"
    },
    "searchOptions":{
        "MiscGroup":{
            "districts":{
                "aggregation":"Aggregate District",
                "districtGroups":{
                    "userGroups":[]
                },
                "districts":[],
                "districtsExpanded":
                    [
                        {
                            "name":"All Districts",
                            "value":"all"
                        }
                    ],
                "districtsSelectType":"all"
            },
            "importPrograms":{
                "aggregation":None,
                "importPrograms":[],
                "programsSelectType":"all"
            },
            "extImportPrograms":{
                "aggregation":"Aggregate CSC",
                "extImportPrograms":[],
                "extImportProgramsExpanded":[],
                "programsSelectType":"all"
            },
            "provisionCodes":{
                "aggregation":"Aggregate RPCODE",
                "provisionCodesSelectType":"all",
                "rateProvisionCodes":[],
                "rateProvisionCodesExpanded":[]
            }
        },
        "commodities":{
            "aggregation":"Aggregate Commodities",
            "codeDisplayFormat":"YES",
            "commodities":[],
            "commoditiesExpanded":[],
            "commoditiesManual":"",
            "commodityGroups":{
                "systemGroups":[],
                "userGroups":[]
            },
            "commoditySelectType":"all",
            "granularity":"2",
            "groupGranularity":None,
            "searchGranularity":None
        },
        "componentSettings":{
            "dataToReport":
                [
                    "CONS_FIR_UNIT_QUANT"
                ],
            "scale":"1",
            "timeframeSelectType":"fullYears",
            "years":
                [
                    "2022","2023"
                ],
            "startDate":None,
            "endDate":None,
            "startMonth":None,
            "endMonth":None,
            "yearsTimeline":"Annual"
        },
        "countries":{
            "aggregation":"Aggregate Countries",
            "countries":[],
            "countriesExpanded":
                [
                    {
                        "name":"All Countries",
                        "value":"all"
                    }
                ],
            "countriesSelectType":"all",
            "countryGroups":{
                "systemGroups":[],
                "userGroups":[]
            }
        }
    },
    "sortingAndDataFormat":{
        "DataSort":{
            "columnOrder":[],
            "fullColumnOrder":[],
            "sortOrder":[]
        },
        "reportCustomizations":{
            "exportCombineTables":False,
            "showAllSubtotal":True,
            "subtotalRecords":"",
            "totalRecords":"20000",
            "exportRawData":False
        }
    }
}
token = os.getenv("USITC_API_KEY")
baseUrl = 'https://datawebws.usitc.gov/dataweb'
headers = {
    "Content-Type": "application/json; charset=utf-8", 
    "Authorization": "Bearer " + token
}
requestData = basicQuery2
response = requests.post(baseUrl+'/api/v2/report2/runReport', 
                         headers=headers, json=requestData, verify=False)

In [2]:
#Helper Functions
#Print Query Function
#This is a simple function designed to be used with the examples above, mostly to avoid copying/pasting the same lines of code over and over again. Note: it is NOT designed to work in most or all cases.

def printQueryResults(headers, requestData):
    response = requests.post(baseUrl+"/api/v2/report2/runReport", 
                            headers=headers, json=requestData, verify=False)

    columns = getColumns(response.json()['dto']['tables'][0]['column_groups'])

    data = getData(response.json()['dto']['tables'][0]['row_groups'][0]['rowsNew'])

    df = pd.DataFrame(data, columns = columns)

    return df
#Support function to extract column names

def getColumns(columnGroups, prevCols = None):
    if prevCols is None:
        columns = []
    else:
        columns = prevCols
    for group in columnGroups:
        if isinstance(group, dict) and 'columns' in group.keys():
            getColumns(group['columns'], columns)
        elif isinstance(group, dict) and 'label' in group.keys():
            columns.append(group['label'])
        elif isinstance(group, list):
            getColumns(group, columns)
    return columns
#Support function to extract data values from JSON

def getData(dataGroups):
    data = []
    for row in dataGroups:
        rowData = []
        for field in row['rowEntries']:
            rowData.append(field['value'])
        data.append(rowData)
    return data

In [3]:
def json_to_python(json_text):
    return json_text.replace('false', 'False').replace('true', 'True').replace('null', 'None')

In [26]:
#This is another basic query
basicQuery2 = {"savedQueryType":"","isOwner":True,"unitConversion":"0","manualConversions":[],"reportOptions":{"tradeType":"Import","classificationSystem":"HTS"},"searchOptions":{"MiscGroup":{"districts":{"aggregation":"Aggregate District","districtGroups":{},"districts":[],"districtsExpanded":[{"name":"All Districts","value":"all"}],"districtsSelectType":"all"},"importPrograms":{"aggregation":None,"importPrograms":[],"programsSelectType":"all"},"extImportPrograms":{"aggregation":"Aggregate CSC","extImportPrograms":[],"extImportProgramsExpanded":[],"programsSelectType":"all"},"provisionCodes":{"aggregation":"Aggregate RPCODE","provisionCodesSelectType":"all","rateProvisionCodes":[],"rateProvisionCodesExpanded":[],"rateProvisionGroups":{"systemGroups":[]}}},"commodities":{"aggregation":"Aggregate Commodities","codeDisplayFormat":"YES","commodities":[],"commoditiesExpanded":[],"commoditiesManual":"","commodityGroups":{"systemGroups":[],"userGroups":[]},"commoditySelectType":"all","granularity":"2","groupGranularity":None,"searchGranularity":None,"showHTSValidDetails":""},"componentSettings":{"dataToReport":["CONS_CUSTOMS_VALUE"],"scale":"1","timeframeSelectType":"customTimePeriod","years":["2023"],"startDate":None,"endDate":None,"startMonth":"01","endMonth":"02","yearsTimeline":"Full Time Period"},"countries":{"aggregation":"Aggregate Countries","countries":[],"countriesExpanded":[{"name":"All Countries","value":"all"}],"countriesSelectType":"all","countryGroups":{"systemGroups":[],"userGroups":[]}}},"sortingAndDataFormat":{"DataSort":{"columnOrder":[],"fullColumnOrder":[],"sortOrder":[]},"reportCustomizations":{"exportCombineTables":False,"totalRecords":"20000","exportRawData":False}},"deletedCountryUserGroups":[],"deletedCommodityUserGroups":[],"deletedDistrictUserGroups":[]}


In [39]:
basicQuery3 = {"savedQueryType":"","isOwner":False,"unitConversion":"0","manualConversions":[],"reportOptions":{"tradeType":"Import","classificationSystem":"HTS"},"searchOptions":{"MiscGroup":{"districts":{"aggregation":"Aggregate District","districtGroups":{},"districts":[],"districtsExpanded":[{"name":"All Districts","value":"all"}],"districtsSelectType":"all"},"importPrograms":{"aggregation":None,"importPrograms":[],"programsSelectType":"all"},"extImportPrograms":{"aggregation":"Aggregate CSC","extImportPrograms":[],"extImportProgramsExpanded":[],"programsSelectType":"all"},"provisionCodes":{"aggregation":"Aggregate RPCODE","provisionCodesSelectType":"all","rateProvisionCodes":[],"rateProvisionCodesExpanded":[],"rateProvisionGroups":{"systemGroups":[]}}},"commodities":{"aggregation":"Break Out Commodities","codeDisplayFormat":"YES","commodities":[],"commoditiesExpanded":[],"commoditiesManual":"","commodityGroups":{"systemGroups":[],"userGroups":[]},"commoditySelectType":"all","granularity":"6","groupGranularity":None,"searchGranularity":None,"showHTSValidDetails":""},"componentSettings":{"dataToReport":["CONS_CUSTOMS_VALUE"],"scale":"1","timeframeSelectType":"customTimePeriod","years":["2025"],"startDate":None,"endDate":None,"startMonth":"01","endMonth":"08","yearsTimeline":"Monthly"},"countries":{"aggregation":"Aggregate Countries","countries":["5330"],"countriesExpanded":[{"name":"India - IN - IND","value":"5330"}],"countriesSelectType":"list","countryGroups":{"systemGroups":[],"userGroups":[]}}},"sortingAndDataFormat":{"DataSort":{"columnOrder":["HTS2 & DESCRIPTION"],"fullColumnOrder":[{"hasChildren":False,"name":"HTS2 & DESCRIPTION","value":"HTS2 & DESCRIPTION","classificationSystem":"","groupUUID":"","items":[],"tradeType":""}],"sortOrder":[{"sortData":"HTS2 & DESCRIPTION","orderBy":"asc","year":""}]},"reportCustomizations":{"exportCombineTables":False,"totalRecords":"20000","exportRawData":False}},"deletedCountryUserGroups":[],"deletedCommodityUserGroups":[],"deletedDistrictUserGroups":[]}

In [4]:
basicQuery4 = json.loads("""{"savedQueryType":"","isOwner":true,"unitConversion":"0","manualConversions":[],"reportOptions":{"tradeType":"Import","classificationSystem":"HTS"},"searchOptions":{"MiscGroup":{"districts":{"aggregation":"Aggregate District","districtGroups":{},"districts":[],"districtsExpanded":[{"name":"All Districts","value":"all"}],"districtsSelectType":"all"},"importPrograms":{"aggregation":null,"importPrograms":[],"programsSelectType":"all"},"extImportPrograms":{"aggregation":"Break Out CSC","extImportPrograms":[],"extImportProgramsExpanded":[],"programsSelectType":"all"},"provisionCodes":{"aggregation":"Break Out RPCODE","provisionCodesSelectType":"all","rateProvisionCodes":[],"rateProvisionCodesExpanded":[],"rateProvisionGroups":{"systemGroups":[]}}},"commodities":{"aggregation":"Break Out Commodities","codeDisplayFormat":"YES","commodities":[],"commoditiesExpanded":[],"commoditiesManual":"","commodityGroups":{"systemGroups":[],"userGroups":[]},"commoditySelectType":"all","granularity":"2","groupGranularity":null,"searchGranularity":null,"showHTSValidDetails":""},"componentSettings":{"dataToReport":["CONS_CUSTOMS_VALUE","CONS_CUSTOMS_VALUE_SUB_DUTY"],"scale":"1","timeframeSelectType":"fullYears","years":["2025"],"startDate":null,"endDate":null,"startMonth":null,"endMonth":null,"yearsTimeline":"Monthly"},"countries":{"aggregation":"Aggregate Countries","countries":[],"countriesExpanded":[{"name":"All Countries","value":"all"}],"countriesSelectType":"all","countryGroups":{"systemGroups":[],"userGroups":[]}}},"sortingAndDataFormat":{"DataSort":{"columnOrder":["RATE-PROVISION-CODE","HTS2 & DESCRIPTION","YEAR","EXTENDED-SPECIAL-IMPORT-PROGRAM"],"fullColumnOrder":[{"hasChildren":false,"name":"Rate Provision Codes","value":"RATE-PROVISION-CODE","classificationSystem":"","groupUUID":"","items":[],"tradeType":""},{"hasChildren":false,"name":"HTS2 & DESCRIPTION","value":"HTS2 & DESCRIPTION","classificationSystem":"","groupUUID":"","items":[],"tradeType":""},{"hasChildren":false,"name":"Year","value":"YEAR","classificationSystem":"","groupUUID":"","items":[],"tradeType":""},{"hasChildren":false,"name":"Special Import Programs","value":"EXTENDED-SPECIAL-IMPORT-PROGRAM","classificationSystem":"","groupUUID":"","items":[],"tradeType":""}],"sortOrder":[{"sortData":"Rate Provision Codes","orderBy":"asc","year":""}]},"reportCustomizations":{"exportCombineTables":false,"totalRecords":"20000","exportRawData":false}},"deletedCountryUserGroups":[],"deletedCommodityUserGroups":[],"deletedDistrictUserGroups":[]}""")

go to Inspect-network-runreport-payload
use json.loads (""" """) to convert string to dictionary


In [30]:
basicQuery5=json.loads("""{"savedQueryType":"","isOwner":true,"unitConversion":"0","manualConversions":[],"reportOptions":{"tradeType":"Import","classificationSystem":"HTS"},"searchOptions":{"MiscGroup":{"districts":{"aggregation":"Aggregate District","districtGroups":{},"districts":[],"districtsExpanded":[{"name":"All Districts","value":"all"}],"districtsSelectType":"all"},"importPrograms":{"aggregation":null,"importPrograms":[],"programsSelectType":"all"},"extImportPrograms":{"aggregation":"Aggregate CSC","extImportPrograms":[],"extImportProgramsExpanded":[],"programsSelectType":"all"},"provisionCodes":{"aggregation":"Aggregate RPCODE","provisionCodesSelectType":"all","rateProvisionCodes":[],"rateProvisionCodesExpanded":[],"rateProvisionGroups":{"systemGroups":[]}}},"commodities":{"aggregation":"Break Out Commodities","codeDisplayFormat":"YES","commodities":[],"commoditiesExpanded":[],"commoditiesManual":"","commodityGroups":{"systemGroups":[],"userGroups":[]},"commoditySelectType":"all","granularity":"4","groupGranularity":null,"searchGranularity":null,"showHTSValidDetails":""},"componentSettings":{"dataToReport":["CONS_CUSTOMS_VALUE"],"scale":"1","timeframeSelectType":"customTimePeriod","years":["2025"],"startDate":null,"endDate":null,"startMonth":"01","endMonth":"07","yearsTimeline":"Monthly"},"countries":{"aggregation":"Aggregate Countries","countries":[],"countriesExpanded":[{"name":"All Countries","value":"all"}],"countriesSelectType":"all","countryGroups":{"systemGroups":[],"userGroups":[]}}},"sortingAndDataFormat":{"DataSort":{"columnOrder":["HTS4 & DESCRIPTION"],"fullColumnOrder":[{"hasChildren":false,"name":"HTS4 & DESCRIPTION","value":"HTS4 & DESCRIPTION","classificationSystem":"","groupUUID":"","items":[],"tradeType":""}],"sortOrder":[{"sortData":"HTS4 & DESCRIPTION","orderBy":"asc","year":""}]},"reportCustomizations":{"exportCombineTables":false,"totalRecords":"20000","exportRawData":false}},"deletedCountryUserGroups":[],"deletedCommodityUserGroups":[],"deletedDistrictUserGroups":[]}""")

In [34]:
basicQuery6=json.loads("""{"savedQueryType":"","isOwner":true,"unitConversion":"0","manualConversions":[],"reportOptions":{"tradeType":"Import","classificationSystem":"HTS"},"searchOptions":{"MiscGroup":{"districts":{"aggregation":"Aggregate District","districtGroups":{},"districts":[],"districtsExpanded":[{"name":"All Districts","value":"all"}],"districtsSelectType":"all"},"importPrograms":{"aggregation":null,"importPrograms":[],"programsSelectType":"all"},"extImportPrograms":{"aggregation":"Aggregate CSC","extImportPrograms":[],"extImportProgramsExpanded":[],"programsSelectType":"all"},"provisionCodes":{"aggregation":"Aggregate RPCODE","provisionCodesSelectType":"all","rateProvisionCodes":[],"rateProvisionCodesExpanded":[],"rateProvisionGroups":{"systemGroups":[]}}},"commodities":{"aggregation":"Break Out Commodities","codeDisplayFormat":"YES","commodities":[],"commoditiesExpanded":[],"commoditiesManual":"","commodityGroups":{"systemGroups":[],"userGroups":[]},"commoditySelectType":"all","granularity":"10","groupGranularity":null,"searchGranularity":null,"showHTSValidDetails":""},"componentSettings":{"dataToReport":["CONS_CUSTOMS_VALUE"],"scale":"1","timeframeSelectType":"customTimePeriod","years":["2025"],"startDate":null,"endDate":null,"startMonth":"01","endMonth":"07","yearsTimeline":"Monthly"},"countries":{"aggregation":"Aggregate Countries","countries":[],"countriesExpanded":[{"name":"All Countries","value":"all"}],"countriesSelectType":"all","countryGroups":{"systemGroups":[],"userGroups":[]}}},"sortingAndDataFormat":{"DataSort":{"columnOrder":["HTS10 & DESCRIPTION"],"fullColumnOrder":[{"hasChildren":false,"name":"HTS10 & DESCRIPTION","value":"HTS10 & DESCRIPTION","classificationSystem":"","groupUUID":"","items":[],"tradeType":""}],"sortOrder":[{"sortData":"HTS10 & DESCRIPTION","orderBy":"asc","year":""}]},"reportCustomizations":{"exportCombineTables":false,"totalRecords":"20000","exportRawData":false}},"deletedCountryUserGroups":[],"deletedCommodityUserGroups":[],"deletedDistrictUserGroups":[]}""")

In [ ]:
query = {"savedQueryType":"","isOwner":true,"unitConversion":"0","manualConversions":[],"reportOptions":{"tradeType":"Import","classificationSystem":"HTS"},"searchOptions":{"MiscGroup":{"districts":{"aggregation":"Aggregate District","districtGroups":{},"districts":[],"districtsExpanded":[{"name":"All Districts","value":"all"}],"districtsSelectType":"all"},"importPrograms":{"aggregation":null,"importPrograms":[],"programsSelectType":"all"},"extImportPrograms":{"aggregation":"Aggregate CSC","extImportPrograms":[],"extImportProgramsExpanded":[],"programsSelectType":"all"},"provisionCodes":{"aggregation":"Aggregate RPCODE","provisionCodesSelectType":"all","rateProvisionCodes":[],"rateProvisionCodesExpanded":[],"rateProvisionGroups":{"systemGroups":[]}}},"commodities":{"aggregation":"Break Out Commodities","codeDisplayFormat":"YES","commodities":[],"commoditiesExpanded":[],"commoditiesManual":"","commodityGroups":{"systemGroups":[],"userGroups":[]},"commoditySelectType":"all","granularity":"10","groupGranularity":null,"searchGranularity":null,"showHTSValidDetails":""},"componentSettings":{"dataToReport":["CONS_CUSTOMS_VALUE"],"scale":"1","timeframeSelectType":"customTimePeriod","years":["2025"],"startDate":null,"endDate":null,"startMonth":"01","endMonth":"07","yearsTimeline":"Monthly"},"countries":{"aggregation":"Aggregate Countries","countries":[],"countriesExpanded":[{"name":"All Countries","value":"all"}],"countriesSelectType":"all","countryGroups":{"systemGroups":[],"userGroups":[]}}},"sortingAndDataFormat":{"DataSort":{"columnOrder":["HTS10 & DESCRIPTION"],"fullColumnOrder":[{"hasChildren":false,"name":"HTS10 & DESCRIPTION","value":"HTS10 & DESCRIPTION","classificationSystem":"","groupUUID":"","items":[],"tradeType":""}],"sortOrder":[{"sortData":"HTS10 & DESCRIPTION","orderBy":"asc","year":""}]},"reportCustomizations":{"exportCombineTables":false,"totalRecords":"20000","exportRawData":false}},"deletedCountryUserGroups":[],"deletedCommodityUserGroups":[],"deletedDistrictUserGroups":[]}

In [ ]:
basicQuery7=json.loads("""query""")

In [35]:
printQueryResults(headers, basicQuery6)

TypeError: 'NoneType' object is not subscriptable